In [3]:
# Import librerias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Para obtener lat y lon
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import json
import numpy as np

# Entregable 2 (parte 1)
## Aprendisaje Automatico

**Equipo:** Patrick Murayari, Bruno Weiss, Juan Jara

**Nombre del proyecto**: Aprender 2016-2024 *Entregable 1*

# Descripción del dataset 


* Frecuencia de actualización: Anualmente
* Responsable: Secretaría de Educación 
* Fecha de publicación: 2016
* Fecha de actualización: 2024
* Fuente primaria: Datos abiertos de la Secretaría de Educación
* https://data.educacion.gob.ar/aprender-respuestas-estudiante


* El formato en el que esta presentado son dos archivos .csv
    * Resultados de lengua
    * Resultados de matematica

Esto inicialmente es sospechoso, por eso iniciamos la limpieza de ambos en simultaneo



Este dataset contiene:

* 44+ variables procedentes del aprender de octubre de 2024
* Datos de Ubicacion
* Resultados de evaluaciones en el area de matematica y lengua

**Variables Categoricas:**

- sector, jurisdiccion, departamento, ambito 


**Variables numéricas:**
- 44 preguntas (dirigidas al alumno) de la encuesta aprender 2024 (ya codificadas one-hot).
- Preguntas sobre el nivel de estudio alcanzado por los padres o adulto responsable.
- Nro_Encuestados: la cantidad de encuestados en esa tupla.

* Conjunto de índices:
- NSE (Nivel Socioeconómico) se construyó con las variables: nivel educativo de madre y padre, hacinamiento, percepción de la Asignación Universal por Hijo
por hogar, acceso a internet y acceso a TICs
- Clima escolar se contruyo con las variables "sin marca" y otros motivos desconocidos. 

**Variable objetivo:**
- desempeños en lengua: 'ldesemp_Satisfactorio', 'ldesemp_Por_debajo_del_nivel_básico', 'ldesemp_Básico', 'ldesemp_Avanzado'.
- desempeños en matematica: 'mdesemp_Por_debajo_del_nivel_básico', 'mdesemp_Satisfactorio', 'mdesemp_Básico', 'mdesemp_Avanzado'.



## Hay muchas preguntas en la encuesta pero a continuacion mostramos la mas relevantes y su forma de contestar.

### Preguntas Relevantes del Cuestionario Estudiantes - Nivel Secundario Aprender 2024


*1. Pregunta sobre el Nivel Educativo de la Familia* 

**Pregunta 1: ¿Cuál fue el último nivel educativo alcanzado por tu mamá, papá o persona adulta responsable?**

*a) No fue a la escuela*

*b) Primario*

*c) Secundario*

*d) Terciario, universitario o posgrado*

---

*2. Pregunta sobre Pertenencia Indígena/Originaria de la Familia*

**Pregunta 2: ¿Tu mamá, papá o persona adulta responsable se reconoce de un pueblo indígena u originario o descendiente de una familia indígena u originaria?por ejemplo: Mapuche, Kolla, Toba, Wichi, Diaguita, Guaraní, Qom, otro)**

*a) Si*

*b) No*

---

*3. Pregunta sobre el Reconocimiento Afrodescendiente de la Familia*

**Pregunta 3: ¿Tu mamá, papá o persona adulta responsable se reconoce afrodescendiente o tiene antepasados negros o africanos?**

*a) Si*

*b) No*

---

*4. Pregunta sobre la Repetición de Grado*

**Pregunta 4: ¿Repetiste de grado alguna vez?, ¿en qué ciclo escolar?**

*a) Primaria*

*b) Secundaria: Ciclo Básico*

*c) Secundaria: Ciclo Orientado*

---

*5. Pregunta sobre las Faltas a Clase*

**Pregunta 5: En lo que va del año, ¿cuántas faltas tenés?**

*a) Ninguna falta*

*b) Menos de 5 faltas*

*c) De 5 a 14 faltas*

*d) De 15 a 19 faltas*

*e) De 20 a 29 faltas*

*f) De 30 o mas faltas*

---

*6. Pregunta sobre la Cantidad de Libros en el Hogar*

**Pregunta 6: Aproximadamente, ¿cuántos libros hay en el lugar donde vivís?**


*a) No hay libros en formato papel*

*b) De 1 a 5 libros*

*c) De 6 a 20 libros*

*d) De 21 a 50 libros*

*e) De 51 a 100 libros*

*f) Más de 100 libros*

---

*7. Pregunta sobre el Acceso a Internet y Computadora*

**Pregunta 7: En donde vivís, ¿hay estos objetos o servicios?**


*a) Servicio de streaming por suscripción (Netflix, Amazon, Disney+, otros)*

*b) Canilla con agua potable (apta para el consumo)*

*c) Lavarropas*

*d) Heladera con freezer*

*e) Televisor*

*f) Celular propio*

*g) Tablet*

*h) Lector de libros digitales (e-book)*

*i) Consola de videojuegos (PlayStation, Nintendo Wii, Xbox, PC Gamer, otras)*

*j) Lector de libros digitales (e-book)*

*k) Consola de videojuegos (PlayStation, Nintendo Wii, Xbox, PC Gamer, otras)*

*l) Baño en el interior del lugar donde vivis*

*m) Conexión a Internet*

---

*8. Pregunta sobre la Convivencia Escolar: Relación con la Comunidad Educativa*

**Pregunta 8: Por lo general, ¿cómo es tu relación con las diferentes personas de la escuela?**

*a) Compañeros de curso*

*b) Profesores*	

*c) Equipo directivo*

---

*9. Pregunta sobre Experiencias de Violencia o Bullying en la Escuela*

**Pregunta 9: Durante este año, ¿viviste alguna de las siguientes situaciones con personas de tu escuela?**

*a) Te molestaron o dejaron de lado*

*b) En redes sociales, te amenazaron, agredieron o dijeron mentiras sobre vos*

*c) De forma física o verbal, te amenazaron, agredieron o dijeron mentiras sobre vos*

*d) Te sacaron tus cosas o las rompieron*

*e) Trataste mal a algún compañero*

*f) Viste alguna situación donde agredieran a algún compañero*


---


*10. Pregunta sobre Proyectos Inmediatos al Finalizar el Secundario*

**Pregunta 10: Al finalizar el secundario, ¿qué proyectos inmediatos tenés?**

*a) Trabajar*

*b) Continuar estudios superiores (universitario o no universitario)*

*c) Hacer cursos de capacitación laboral (electricidad, peluquería, manicuría, de huerta, artesanías, del área de construcciones, etc.)*

*d) Ayudar a mi familia con las tareas del hogar*

*e) Producir contenidos para las redes sociales*

*f) Jugar videojuegos para generar dinero*

*g) Realizar trabajos solidarios o comunitarios (participación en comedores, asistencia a personas mayores, acciones religiosas, etc.)*

*h) Desarrollar un emprendimiento personal o familiar*

*i) Desarrollar otras actividades (viajar, hacer deportes, actividades artísticas, etc.)*

*j) Aún no lo decidí*

# Declaracion de ubicaciones de archivos:
- esto es para que no queden desordenados

In [5]:
ruta_archivo_leng = "../data/raw/2019 Base APRENDER - Censal - Secundaria 5-6 año - Agregada - Desempeños de Lengua(in).csv"
ruta_archivo_mate = "../data/raw/2019 Base APRENDER - Censal - Secundaria 5-6 año - Agregada - Desempeños de Matematica(in).csv"

ruta_archivo_leng_pre = "../data/interim/2019_Ap_56Lengcoma.csv"
ruta_archivo_mate_pre = "../data/interim/2019_Ap_56Matecoma.csv"

ruta_archivo_leng_pre2 = "../data/interim/2019_Ap_56Lengceros.csv"
ruta_archivo_mate_pre2 = "../data/interim/2019_Ap_56Mateceros.csv"

ruta_archivo_final = "../data/interim/2019_Ap_56listo.csv"
ruta_archivo_final_latlon = "../data/interim/2019_Ap_56latlon.csv"
ruta_archivo_final_final = "../data/processed/2019_Ap_56final.csv"


# Recodificacion 
* Debido a que el dataset tiene una codificacion latin-1 y
  utiliza el coma como punto decimal, esta funcion reemplaza por punto las comas

In [3]:
def reemplazar_comas_por_puntos(input_csv, output_csv):
    """
    Reemplaza todas las comas por puntos en un archivo CSV con separador ';'
    y guarda el resultado en un nuevo archivo.
    
    :param input_csv: Ruta del archivo CSV original
    :param output_csv: Ruta del archivo CSV de salida
    """
    with open(input_csv, 'r', encoding='latin-1') as f_in:
        contenido = f_in.read()
    
    # Reemplaza todas las comas por puntos
    contenido = contenido.replace(',', '.')
    
    with open(output_csv, 'w', encoding='utf-8') as f_out:
        f_out.write(contenido)

In [4]:
reemplazar_comas_por_puntos(ruta_archivo_leng, ruta_archivo_leng_pre)
reemplazar_comas_por_puntos(ruta_archivo_mate, ruta_archivo_mate_pre)

* Se abre el nuevo archivo generado y se comprueba su estado

In [5]:
df1 = pd.read_csv(ruta_archivo_leng_pre, sep=';', decimal=".", low_memory=False)
df2 = pd.read_csv(ruta_archivo_mate_pre, sep=';', decimal=".", low_memory=False)

In [6]:
df1.head(2)

,cod_provincia,departamento,sector,ambito,ap01_01_Blanco,ap01_01_No_disponible,ap01_01_Multimarca,ap01_01_No_Corresponde,ap01_01_Enero,ap01_01_Febrero,...,iinfraestructura_Medio,iinfraestructura_Alto,ap42_01rec_Siempre_La_mayoría_de_las_veces,ap42_01rec_Algunas_veces_Nunca,ap42_02rec_Siempre_La_mayoría_de_las_veces,ap42_02rec_Algunas_veces_Nunca,ap42_03rec_Siempre_La_mayoría_de_las_veces,ap42_03rec_Algunas_veces_Nunca,ap42_04rec_Siempre_La_mayoría_de_las_veces,ap42_04rec_Algunas_veces_Nunca
0,Ciudad Autónoma de Buenos Aires,Comuna 1,Estatal,Urbano,42.0012506,,3.5640177,,123.125973,126.2559576,...,485.7583492,,687.3216511,507.1178015,249.081601,931.8155816,241.8413646,929.0444838,271.4511898,906.1580227
1,Ciudad Autónoma de Buenos Aires,Comuna 1,Privado,Urbano,12.8345277,9.7096522,1.9187832,,110.3910931,107.8387444,...,831.4726977,223.8580275,352.9426799,933.2570792,287.3229386,989.949793,259.0979094,1012.498422,282.2082383,987.9490059


* Como se observa que hay columnas vacias, se prodece a procesarlas para ponerlas 
 con el valor 0 de tipo float, dejando sin cambios las primeras 4 columnas que son categoricas

In [7]:
df1.columns

Index(['cod_provincia', 'departamento', 'sector', 'ambito', 'ap01_01_Blanco',
       'ap01_01_No_disponible', 'ap01_01_Multimarca', 'ap01_01_No_Corresponde',
       'ap01_01_Enero', 'ap01_01_Febrero',
       ...
       'iinfraestructura_Medio', 'iinfraestructura_Alto',
       'ap42_01rec_Siempre_La_mayoría_de_las_veces',
       'ap42_01rec_Algunas_veces_Nunca',
       'ap42_02rec_Siempre_La_mayoría_de_las_veces',
       'ap42_02rec_Algunas_veces_Nunca',
       'ap42_03rec_Siempre_La_mayoría_de_las_veces',
       'ap42_03rec_Algunas_veces_Nunca',
       'ap42_04rec_Siempre_La_mayoría_de_las_veces',
       'ap42_04rec_Algunas_veces_Nunca'],
      dtype='object', length=1414)

In [8]:
def rellenar_y_convertir_columnas_desde_4(df: pd.DataFrame) -> pd.DataFrame:
    """
    Rellena los valores vacíos o con solo espacios con 0 y convierte a float
    todas las columnas desde la 4ta hasta la última del DataFrame.
    Evita el warning de downcasting de pandas.
    """
    columnas = df.columns[4:]
    
    for col in columnas:
        df[col] = df[col].mask(df[col].astype(str).str.strip() == '', 0)
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(float)
    
    return df

In [9]:
df1 = rellenar_y_convertir_columnas_desde_4(df1)
df2 = rellenar_y_convertir_columnas_desde_4(df2)

In [10]:
df1.head()

,cod_provincia,departamento,sector,ambito,ap01_01_Blanco,ap01_01_No_disponible,ap01_01_Multimarca,ap01_01_No_Corresponde,ap01_01_Enero,ap01_01_Febrero,...,iinfraestructura_Medio,iinfraestructura_Alto,ap42_01rec_Siempre_La_mayoría_de_las_veces,ap42_01rec_Algunas_veces_Nunca,ap42_02rec_Siempre_La_mayoría_de_las_veces,ap42_02rec_Algunas_veces_Nunca,ap42_03rec_Siempre_La_mayoría_de_las_veces,ap42_03rec_Algunas_veces_Nunca,ap42_04rec_Siempre_La_mayoría_de_las_veces,ap42_04rec_Algunas_veces_Nunca
0,Ciudad Autónoma de Buenos Aires,Comuna 1,Estatal,Urbano,42.001251,0.000000,3.564018,0.0,123.125973,126.255958,...,485.758349,0.000000,687.321651,507.117801,249.081601,931.815582,241.841365,929.044484,271.451190,906.158023
1,Ciudad Autónoma de Buenos Aires,Comuna 1,Privado,Urbano,12.834528,9.709652,1.918783,0.0,110.391093,107.838744,...,831.472698,223.858027,352.942680,933.257079,287.322939,989.949793,259.097909,1012.498422,282.208238,987.949006
2,Ciudad Autónoma de Buenos Aires,Comuna 10,Estatal,Urbano,10.181909,2.794852,0.000000,0.0,69.231848,87.552754,...,795.867239,0.000000,509.652602,267.563095,195.390496,570.279806,183.276339,583.174344,177.430295,583.703948
3,Ciudad Autónoma de Buenos Aires,Comuna 10,Privado,Urbano,15.489986,2.452990,3.208084,0.0,103.684723,104.976365,...,979.277174,0.000000,41.937935,1079.071916,74.829289,1042.663956,143.076679,966.261530,136.262966,975.641386
4,Ciudad Autónoma de Buenos Aires,Comuna 11,Estatal,Urbano,4.043269,4.645987,0.000000,0.0,45.018740,36.392928,...,435.742750,66.060125,271.721618,275.720897,103.445483,437.423594,126.636843,415.163944,149.130616,393.279957


In [11]:
df2.head()

,cod_provincia,departamento,sector,ambito,ap01_01_Blanco,ap01_01_No_disponible,ap01_01_Multimarca,ap01_01_No_Corresponde,ap01_01_Enero,ap01_01_Febrero,...,iinfraestructura_Medio,iinfraestructura_Alto,ap42_01rec_Siempre_La_mayoría_de_las_veces,ap42_01rec_Algunas_veces_Nunca,ap42_02rec_Siempre_La_mayoría_de_las_veces,ap42_02rec_Algunas_veces_Nunca,ap42_03rec_Siempre_La_mayoría_de_las_veces,ap42_03rec_Algunas_veces_Nunca,ap42_04rec_Siempre_La_mayoría_de_las_veces,ap42_04rec_Algunas_veces_Nunca
0,Ciudad Autónoma de Buenos Aires,Comuna 1,Estatal,Urbano,34.198954,0.000000,10.649127,0.0,125.390912,128.687450,...,494.732487,0.000000,700.747215,539.705264,252.466741,975.244191,258.425187,972.861605,280.800877,958.189401
1,Ciudad Autónoma de Buenos Aires,Comuna 1,Privado,Urbano,12.812932,9.828451,1.915555,0.0,107.813051,109.264032,...,830.073622,223.481361,355.344015,934.637293,287.970824,992.921812,265.849733,1009.376054,286.178537,987.610584
2,Ciudad Autónoma de Buenos Aires,Comuna 10,Estatal,Urbano,11.487818,1.164931,0.000000,0.0,79.809261,83.862387,...,798.403030,0.000000,514.699878,269.803591,204.554959,566.505782,190.350905,580.138836,179.543755,583.205649
3,Ciudad Autónoma de Buenos Aires,Comuna 10,Privado,Urbano,15.567032,2.448862,0.000000,0.0,106.078492,101.571363,...,976.013047,0.000000,41.238338,1071.727817,78.501993,1031.088500,143.385887,957.322651,133.002668,971.008336
4,Ciudad Autónoma de Buenos Aires,Comuna 11,Estatal,Urbano,4.031603,0.000000,0.000000,0.0,51.090897,45.052263,...,436.266771,66.255468,275.649020,268.270972,108.626643,429.124415,127.877210,406.386751,152.412930,385.133866


# Guardado intermedio
* Se crea esta funcion para guardar los .csv

In [38]:
def guardar_df(df: pd.DataFrame, ruta: str):
    """
    Guarda un DataFrame en un archivo CSV en la ruta indicada.

    Parámetros:
    df (pd.DataFrame): DataFrame a guardar.
    ruta (str): Ruta completa del archivo donde se guardará el CSV.
    """
    try:
        df.to_csv(ruta, index=False, encoding='utf-8')
        print(f"Archivo guardado exitosamente en: {ruta}")
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")

In [13]:
guardar_df(df1, ruta_archivo_leng_pre2)
guardar_df(df2, ruta_archivo_mate_pre2)

Archivo guardado exitosamente en: ../data/interim/2019_Ap_56Lengceros.csv
Archivo guardado exitosamente en: ../data/interim/2019_Ap_56Mateceros.csv


In [7]:
#Reapertura de los archivos
df1 = pd.read_csv(ruta_archivo_leng_pre2, low_memory=False)
df2 = pd.read_csv(ruta_archivo_mate_pre2, low_memory=False)

In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1178 entries, 0 to 1177
Columns: 1414 entries, cod_provincia to ap42_04rec_Algunas_veces_Nunca
dtypes: float64(1410), object(4)
memory usage: 12.7+ MB


In [9]:
df1.dtypes

cod_provincia                                  object
departamento                                   object
sector                                         object
ambito                                         object
ap01_01_Blanco                                float64
                                               ...   
ap42_02rec_Algunas_veces_Nunca                float64
ap42_03rec_Siempre_La_mayoría_de_las_veces    float64
ap42_03rec_Algunas_veces_Nunca                float64
ap42_04rec_Siempre_La_mayoría_de_las_veces    float64
ap42_04rec_Algunas_veces_Nunca                float64
Length: 1414, dtype: object

## Se procede a comparar ambos dataset el de lengua y matematica

In [10]:
# Comparar dimensiones (deberían ser iguales)
print("Shape archivo1:", df1.shape)
print("Shape archivo2:", df2.shape)

Shape archivo1: (1178, 1414)
Shape archivo2: (1178, 1414)


In [11]:
print(df1.columns)

Index(['cod_provincia', 'departamento', 'sector', 'ambito', 'ap01_01_Blanco',
       'ap01_01_No_disponible', 'ap01_01_Multimarca', 'ap01_01_No_Corresponde',
       'ap01_01_Enero', 'ap01_01_Febrero',
       ...
       'iinfraestructura_Medio', 'iinfraestructura_Alto',
       'ap42_01rec_Siempre_La_mayoría_de_las_veces',
       'ap42_01rec_Algunas_veces_Nunca',
       'ap42_02rec_Siempre_La_mayoría_de_las_veces',
       'ap42_02rec_Algunas_veces_Nunca',
       'ap42_03rec_Siempre_La_mayoría_de_las_veces',
       'ap42_03rec_Algunas_veces_Nunca',
       'ap42_04rec_Siempre_La_mayoría_de_las_veces',
       'ap42_04rec_Algunas_veces_Nunca'],
      dtype='object', length=1414)


In [17]:
# Listar las columnas de cada uno
cols1 = set(df1.columns)
cols2 = set(df2.columns)

#print("Columnas en archivo1:", cols1)
#print("Columnas en archivo2:", cols2)

# Columnas que comparten
comunes = cols1.intersection(cols2)
#print("\n✅ Columnas comunes:", comunes)

# Columnas que están en uno pero no en el otro
solo_en_1 = cols1 - cols2
solo_en_2 = cols2 - cols1

print("\n📌 Solo en archivo1:", solo_en_1)
print("📌 Solo en archivo2:", solo_en_2)


📌 Solo en archivo1: {'isocioal_Medio', 'isocioal_Bajo', 'ldesemp_Básico', 'ldesemp_Por_debajo_del_nivel_básico', 'isocioal_Alto', 'ldesemp_Avanzado', 'ldesemp_Satisfactorio'}
📌 Solo en archivo2: {'isocioam_Alto', 'mdesemp_Satisfactorio', 'isocioam_Medio', 'mdesemp_Por_debajo_del_nivel_básico', 'mdesemp_Avanzado', 'mdesemp_Básico', 'isocioam_Bajo'}


# Variables encontradas que se van a dropear
📌 Solo en archivo1: {'isocioal_Medio','isocioal_Alto','isocioal_Bajo'}
📌 Solo en archivo2: {'isocioam_Alto', 'isocioam_Bajo','isocioam_Medio}

In [18]:
df1 = df1.drop(columns=['isocioal_Alto', 'isocioal_Bajo','isocioal_Medio'])
df2 = df2.drop(columns=['isocioam_Alto', 'isocioam_Bajo','isocioam_Medio'])

# Diferencias encontradas

hay columnas que estan en uno de ambos datasets, por lo que le vamos a hacer un join


Pero antes, vamos a ver si difieren en ambos datos


In [19]:
# Columnas comunes (convertir a lista ordenada)
comunes = sorted(set(df1.columns).intersection(set(df2.columns)))


# Filtrar ambos dataframes solo con las columnas comunes
df1 = df1[comunes]
df2 = df2[comunes]

# Ver en qué filas/columnas difieren
diff = (df1 != df2)

# Contar cuántas celdas son distintas
total_diferencias = diff.sum().sum()
#print("Total de diferencias:", total_diferencias)

# Mostrar las filas con diferencias
diferencias = df1[diff.any(axis=1)]
#print("Filas con diferencias:")
#print(diferencias)

# Si querés ver lado a lado:
comparacion = df1.compare(df2)
print("Comparacion")
print(comparacion.head(3))


Comparacion
  NSE_quintiles_Q1             NSE_quintiles_Q2             NSE_quintiles_Q3  \
              self       other             self       other             self   
0       177.033207  167.068371       297.156926  301.616881       167.260748   
1        50.485552   52.359233       140.262320  140.299925       170.422234   
2       113.981402  108.509944       144.102619  146.143124       194.796944   

              NSE_quintiles_Q4             NSE_quintiles_Q5              ...  \
        other             self       other             self       other  ...   
0  190.696428       219.564204  204.324652       486.595062  517.233900  ...   
1  164.713360       242.684706  248.925140       702.905071  696.495603  ...   
2  201.122303       208.279365  207.619009       140.479978  140.215626  ...   

  trabaja_fuera_hogar_NO              trabaja_fuera_hogar_SI              \
                    self        other                   self       other   
0            1008.745736  1039.810

# Vista en profundidad

debido a que ambos dataset tinen datos censurados, vamos a ver que tanto difieren

la solucion sera hacer un promedio de las columnas que son iguales


In [20]:
# Supongamos que tus DataFrames se llaman df1 y df2

# Columnas comunes
columnas_comunes = df1.columns.intersection(df2.columns)

# Excluir columnas de tipo object (strings)
columnas_numericas = [col for col in columnas_comunes if pd.api.types.is_numeric_dtype(df1[col])]

# Seleccionamos solo columnas numéricas y hacemos copia
df1_comun = df1[columnas_numericas].copy()
df2_comun = df2[columnas_numericas].copy()

# Convertir números con coma decimal a float
for col in columnas_numericas:
    df1_comun[col] = df1_comun[col].astype(str).str.replace(',', '.').astype(float)
    df2_comun[col] = df2_comun[col].astype(str).str.replace(',', '.').astype(float)

# Restar DataFrames
diferencias = df1_comun - df2_comun

# Estadísticas
pd.set_option('display.max_rows', None)
print(diferencias.describe())
print("Máxima diferencia por columna:\n", diferencias.max())
#print("Máxima diferencia absoluta total:", diferencias.abs().max().max())

       NSE_quintiles_Q1  NSE_quintiles_Q2  NSE_quintiles_Q3  NSE_quintiles_Q4  \
count       1178.000000       1178.000000       1178.000000       1178.000000   
mean           0.047100         -0.170340         -0.192279         -0.155942   
std            4.104383          3.264213          3.431780          2.496982   
min          -36.498736        -32.463882        -29.995516        -29.283515   
25%           -0.541803         -0.564100         -0.576787         -0.415842   
50%            0.000000         -0.000813         -0.004874          0.000000   
75%            0.451317          0.362685          0.243764          0.272524   
max           96.477414         47.403379         68.532184         21.276336   

       NSE_quintiles_Q5  ap01_01_Abril  ap01_01_Agosto  ap01_01_Blanco  \
count       1178.000000    1178.000000     1178.000000     1178.000000   
mean          -0.289614      -0.019653       -0.120380        0.123403   
std            2.712370       2.094628        1.

In [21]:

recuento = ["ap01_01_Abril","ap01_01_Agosto","ap01_01_Blanco","ap01_01_Diciembre",
            "ap01_01_Enero","ap01_01_Febrero","ap01_01_Julio","ap01_01_Junio",
            "ap01_01_Marzo","ap01_01_Mayo","ap01_01_Multimarca", "ap01_01_No_disponible",
            "ap01_01_Noviembre","ap01_01_Octubre","ap01_01_Septiembre"]



df1['Nro_Encuestados'] = df1[recuento].sum(axis=1)
df2['Nro_Encuestados'] = df2[recuento].sum(axis=1)

In [22]:
print(df1['Nro_Encuestados'])
print(df2['Nro_Encuestados'])

0       1517.210099
1       1377.727426
2        879.901023
3       1178.970168
4        623.423168
5       1107.610114
6        610.672227
7       1410.964563
8        875.686568
9       1721.695167
10       636.565061
11      1047.612567
12       510.845854
13       657.814647
14       635.981425
15       558.765681
16      1132.069681
17       963.195886
18      1153.830164
19       803.010762
20       724.806162
21      1474.836478
22       870.977333
23       694.129416
24       951.958094
25       917.547343
26      1147.120474
27       742.281284
28      1041.701172
29      1210.537157
30         6.967118
31        80.396062
32       280.637867
33        93.547267
34        64.605412
35       307.748973
36       132.729690
37        27.819148
38       136.040004
39        23.810926
40        38.377351
41        24.884913
42        57.893070
43      3628.183066
44      2617.492312
45        19.835590
46       220.295552
47      2166.759084
48      1833.768878
49        27.769824


#Esto se hace una sola vez

In [23]:
# Excluir columnas de tipo object (strings)

columnas_numericas = df1.columns.drop(["cod_provincia","departamento","sector","ambito","Nro_Encuestados"]).tolist()
df1[columnas_numericas] = df1[columnas_numericas].div(df1["Nro_Encuestados"], axis=0)

columnas_numericas = df2.columns.drop(["cod_provincia","departamento","sector","ambito","Nro_Encuestados"]).tolist()
df2[columnas_numericas] = df2[columnas_numericas].div(df2["Nro_Encuestados"], axis=0)

In [24]:
df1.head()

,NSE_quintiles_Q1,NSE_quintiles_Q2,NSE_quintiles_Q3,NSE_quintiles_Q4,NSE_quintiles_Q5,ambito,ap01_01_Abril,ap01_01_Agosto,ap01_01_Blanco,ap01_01_Diciembre,...,sobreedad_Edad_teórica_para_el_grado_17_años_al_30_de_junio,sobreedad_Menos_de_17_años,trabaja_fuera_hogar_Blanco,trabaja_fuera_hogar_NO,trabaja_fuera_hogar_SI,trabaja_fuera_hogar_remunerado_Blanco,trabaja_fuera_hogar_remunerado_No_trabaja_fuera_del_hogar,trabaja_fuera_hogar_remunerado_remuneradamente,trabaja_fuera_hogar_remunerado_sin_remuneración,Nro_Encuestados
0,0.116683,0.195857,0.110242,0.144716,0.320717,Urbano,0.083848,0.082116,0.027683,0.096008,...,0.539129,0.011530,0.0,0.664869,0.252364,0.0,0.664869,0.216411,0.035953,1517.210099
1,0.036644,0.101807,0.123698,0.176149,0.510192,Urbano,0.090951,0.068534,0.009316,0.076745,...,0.789528,0.002392,0.0,0.767802,0.210633,0.0,0.767802,0.188643,0.021989,1377.727426
2,0.129539,0.163771,0.221385,0.236708,0.159654,Urbano,0.058170,0.062070,0.011572,0.072926,...,0.575496,0.007125,0.0,0.711721,0.249621,0.0,0.711721,0.203237,0.046384,879.901023
3,0.011646,0.068201,0.191922,0.270649,0.408814,Urbano,0.132744,0.093142,0.013139,0.048393,...,0.823423,0.009725,0.0,0.805254,0.183318,0.0,0.805254,0.128512,0.054806,1178.970168
4,0.115283,0.191566,0.246344,0.233943,0.117339,Urbano,0.107958,0.082251,0.006486,0.084176,...,0.597906,0.016166,0.0,0.683337,0.263942,0.0,0.683337,0.233603,0.030339,623.423168


In [25]:
df2.head()

,NSE_quintiles_Q1,NSE_quintiles_Q2,NSE_quintiles_Q3,NSE_quintiles_Q4,NSE_quintiles_Q5,ambito,ap01_01_Abril,ap01_01_Agosto,ap01_01_Blanco,ap01_01_Diciembre,...,sobreedad_Edad_teórica_para_el_grado_17_años_al_30_de_junio,sobreedad_Menos_de_17_años,trabaja_fuera_hogar_Blanco,trabaja_fuera_hogar_NO,trabaja_fuera_hogar_SI,trabaja_fuera_hogar_remunerado_Blanco,trabaja_fuera_hogar_remunerado_No_trabaja_fuera_del_hogar,trabaja_fuera_hogar_remunerado_remuneradamente,trabaja_fuera_hogar_remunerado_sin_remuneración,Nro_Encuestados
0,0.107891,0.194781,0.123150,0.131951,0.334024,Urbano,0.082810,0.080623,0.022085,0.092958,...,0.549520,0.019861,0.0,0.671498,0.256145,0.0,0.671498,0.214373,0.041772,1548.494658
1,0.038068,0.102006,0.119756,0.180983,0.506392,Urbano,0.093969,0.067439,0.009316,0.073579,...,0.784570,0.002392,0.0,0.773593,0.207659,0.0,0.773593,0.189561,0.018098,1375.409188
2,0.123033,0.165703,0.228041,0.235407,0.158982,Urbano,0.066233,0.062109,0.013025,0.070526,...,0.561497,0.006997,0.0,0.716499,0.246038,0.0,0.716499,0.199152,0.046886,881.956389
3,0.010049,0.067010,0.194533,0.267199,0.412218,Urbano,0.132490,0.094321,0.013240,0.048894,...,0.822003,0.009682,0.0,0.801151,0.189266,0.0,0.801151,0.136173,0.053093,1175.768964
4,0.102861,0.185533,0.240980,0.254537,0.119435,Urbano,0.094350,0.084213,0.006444,0.084040,...,0.617939,0.016157,0.0,0.697589,0.256598,0.0,0.697589,0.224823,0.031776,625.615887


# Al pasarlo a frecuencias vamos a ver si hay diferencias

In [26]:
# Columnas comunes
columnas_comunes = df1.columns.intersection(df2.columns)

# Excluir columnas de tipo object (strings)
columnas_numericas = [col for col in columnas_comunes if pd.api.types.is_numeric_dtype(df1[col])]

# Seleccionamos solo columnas numéricas y hacemos copia
df1_comun = df1[columnas_numericas].copy()
df2_comun = df2[columnas_numericas].copy()

# Convertir números con coma decimal a float
for col in columnas_numericas:
    df1_comun[col] = df1_comun[col].astype(str).str.replace(',', '.').astype(float)
    df2_comun[col] = df2_comun[col].astype(str).str.replace(',', '.').astype(float)

# Restar DataFrames
diferencias = df1_comun - df2_comun

# Estadísticas
pd.set_option('display.max_rows', None)
#print(diferencias.describe())
#print("Máxima diferencia por columna:\n", diferencias.max())
#print("Máxima diferencia absoluta total:", diferencias.abs().max().max())

In [27]:
print("Máxima diferencia absoluta total:", diferencias.abs().max().max())

Máxima diferencia absoluta total: 224.54031740000028


* la columna ap22c_Entre_5_y_10_horas_en_la_semana varia un 36% entre ambos data sets




sabiendo que en ambos datasets hay datos disertantes debido a un proceso de 
proteccion de datos personales, vamos a utilizar el promedio para unificar un dataset

In [30]:
#Reapertura de los archivos
df1 = pd.read_csv(ruta_archivo_leng_pre2, low_memory=False)
df2 = pd.read_csv(ruta_archivo_mate_pre2, low_memory=False)


In [31]:


recuento = ["ap01_01_Abril","ap01_01_Agosto","ap01_01_Blanco","ap01_01_Diciembre",
            "ap01_01_Enero","ap01_01_Febrero","ap01_01_Julio","ap01_01_Junio",
            "ap01_01_Marzo","ap01_01_Mayo","ap01_01_Multimarca", "ap01_01_No_disponible",
            "ap01_01_Noviembre","ap01_01_Octubre","ap01_01_Septiembre"]

df1['Nro_Encuestados'] = df1[recuento].sum(axis=1)
df2['Nro_Encuestados'] = df2[recuento].sum(axis=1)



#df1['ldesemp_Por_debajo_del_nivel_básico'] = df1['ldesemp_Por_debajo_del_básico']
#df1 = df1.drop(columns='ldesemp_Por_debajo_del_básico')
#df2['mdesemp_Por_debajo_del_nivel_básico'] = df2['mdesemp_Por_debajo_del_básico']
#df2 = df2.drop(columns='mdesemp_Por_debajo_del_básico')

Sena1 = ['ldesemp_Avanzado', 'ldesemp_Por_debajo_del_nivel_básico',
                      'ldesemp_Básico', 'ldesemp_Satisfactorio']
Sena2 = [ 'mdesemp_Básico', 'mdesemp_Por_debajo_del_nivel_básico',
                    'mdesemp_Satisfactorio', 'mdesemp_Avanzado']
df1[Sena1] = df1[Sena1].div(df1["Nro_Encuestados"], axis=0)

df2[Sena2] = df2[Sena2].div(df2["Nro_Encuestados"], axis=0)


In [32]:
df1[Sena1].head(3)

,ldesemp_Avanzado,ldesemp_Por_debajo_del_nivel_básico,ldesemp_Básico,ldesemp_Satisfactorio
0,0.283612,0.215129,0.167123,0.399530
1,0.402024,0.073894,0.101015,0.426984
2,0.200280,0.132008,0.198444,0.486380


In [33]:
# Columnas a excluir de df1
excluir = ["cod_provincia","departamento","sector","ambito"]

# Columnas comunes (sin las excluidas)
comunes = [c for c in df1.columns.intersection(df2.columns) if c not in excluir]

# Promedio de las comunes
promedio = (df1[comunes] + df2[comunes]) / 2

# Columnas exclusivas de df2
solo_df2 = df2[Sena2]
solo_df1 = df1[Sena1]
exclu = df1[excluir]

# Unificación final: comunes promediadas + exclusivas de df2
resultado = pd.concat([promedio, solo_df2, solo_df1, exclu], axis=1)



In [34]:
resultado.head(3)

,ap01_01_Blanco,ap01_01_No_disponible,ap01_01_Multimarca,ap01_01_No_Corresponde,ap01_01_Enero,ap01_01_Febrero,ap01_01_Marzo,ap01_01_Abril,ap01_01_Mayo,ap01_01_Junio,...,mdesemp_Satisfactorio,mdesemp_Avanzado,ldesemp_Avanzado,ldesemp_Por_debajo_del_nivel_básico,ldesemp_Básico,ldesemp_Satisfactorio,cod_provincia,departamento,sector,ambito
0,38.100103,0.000000,7.106572,0.0,124.258442,127.471704,124.401839,127.723430,106.790121,129.026481,...,0.327290,0.150694,0.283612,0.215129,0.167123,0.399530,Ciudad Autónoma de Buenos Aires,Comuna 1,Estatal,Urbano
1,12.823730,9.769051,1.917169,0.0,109.102072,108.551388,99.718282,127.275887,125.298866,129.640379,...,0.429052,0.219515,0.402024,0.073894,0.101015,0.426984,Ciudad Autónoma de Buenos Aires,Comuna 1,Privado,Urbano
2,10.834863,1.979891,0.000000,0.0,74.520555,85.707571,101.649904,54.799012,60.765202,76.470780,...,0.437338,0.069351,0.200280,0.132008,0.198444,0.486380,Ciudad Autónoma de Buenos Aires,Comuna 10,Estatal,Urbano


In [35]:

#Recuperando los valores
resultado[Sena1] = resultado[Sena1].mul(resultado["Nro_Encuestados"], axis=0)
resultado[Sena2] = resultado[Sena2].mul(resultado["Nro_Encuestados"], axis=0)



In [36]:
resultado.head()

,ap01_01_Blanco,ap01_01_No_disponible,ap01_01_Multimarca,ap01_01_No_Corresponde,ap01_01_Enero,ap01_01_Febrero,ap01_01_Marzo,ap01_01_Abril,ap01_01_Mayo,ap01_01_Junio,...,mdesemp_Satisfactorio,mdesemp_Avanzado,ldesemp_Avanzado,ldesemp_Por_debajo_del_nivel_básico,ldesemp_Básico,ldesemp_Satisfactorio,cod_provincia,departamento,sector,ambito
0,38.100103,0.000000,7.106572,0.0,124.258442,127.471704,124.401839,127.723430,106.790121,129.026481,...,501.687133,230.992331,434.734796,329.761717,256.174319,612.420663,Ciudad Autónoma de Buenos Aires,Comuna 1,Estatal,Urbano
1,12.823730,9.769051,1.917169,0.0,109.102072,108.551388,99.718282,127.275887,125.298866,129.640379,...,590.619286,302.177951,553.412811,101.720018,139.053569,587.772806,Ciudad Autónoma de Buenos Aires,Comuna 1,Privado,Urbano
2,10.834863,1.979891,0.000000,0.0,74.520555,85.707571,101.649904,54.799012,60.765202,76.470780,...,385.263801,61.093518,176.431993,116.290074,174.814780,428.465844,Ciudad Autónoma de Buenos Aires,Comuna 10,Estatal,Urbano
3,15.528509,2.450926,1.604042,0.0,104.881607,103.273864,89.211479,156.139603,75.809453,79.649385,...,620.876357,82.528744,378.052771,86.016384,154.661004,563.029198,Ciudad Autónoma de Buenos Aires,Comuna 10,Privado,Urbano
4,4.037436,2.322993,0.000000,0.0,48.054818,40.722596,42.911736,63.165032,70.551349,48.007761,...,209.683760,13.211296,110.159428,96.242425,122.404094,308.268201,Ciudad Autónoma de Buenos Aires,Comuna 11,Estatal,Urbano


In [39]:
guardar_df(resultado, ruta_archivo_final)

Archivo guardado exitosamente en: ../data/interim/2019_Ap_56listo.csv


# Transformaciones: 

* Pasamos las columnas jurisdiccion y departamento a coordenadas geograficas
    * Esto es para no tener que hacer codificaciones onehot
    * Tambien sirve para poder introducir directamente la ubicacion del establecimiento educativo.


* Para lograr esto, utilizamos la api de Nominatim
    * Las consultas son cada 5 segundos (86 minutos)
    * Se genera un json para usarlo de cache.
    * Los departamentos enmascarados van a tener el centroide de la provincia
    * Los departamentos que no se encuentren, tambien van a tener el centroide de la provincia. 

In [40]:
#Abrimos el ultimo archivo
df = pd.read_csv(ruta_archivo_final, low_memory=False)

In [41]:
df['cod_provincia'].head(3)

0    Ciudad Autónoma de Buenos Aires
1    Ciudad Autónoma de Buenos Aires
2    Ciudad Autónoma de Buenos Aires
Name: cod_provincia, dtype: object

In [44]:

# Inicializar geolocalizador
geolocator = Nominatim(user_agent="mi_app_geocoding")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=5)

# Cargar cache si existe
try:
    with open('cache_geocoding.json', 'r') as f:
        cache = json.load(f)
except FileNotFoundError:
    cache = {}

# Función para obtener coordenadas
def obtener_coordenadas(juris, depar):
    # Si es Enmascadaro, usar solo la provincia
    if depar == "Enmascarado":
        key = f"{juris}"  # solo la provincia
    else:
        key = f"{depar}, {juris}"  # combinación normal

    if key in cache:
        return pd.Series(cache[key])
    
    try:
        location = geocode(key)
        if location:
            coords = [location.latitude, location.longitude]
            cache[key] = coords
            return pd.Series(coords)
        else:
            cache[key] = [None, None]
            return pd.Series([None, None])
    except:
        cache[key] = [None, None]
        return pd.Series([None, None])

# Aplicar función
df[['lat', 'lon']] = df.apply(lambda row: obtener_coordenadas(row['cod_provincia'], row['departamento']), axis=1)

# Guardar cache en disco
with open('cache_geocoding.json', 'w') as f:
    json.dump(cache, f)


In [45]:
guardar_df(df, ruta_archivo_final_latlon)

Archivo guardado exitosamente en: ../data/interim/2019_Ap_56latlon.csv


- confirmamos que en las ultimas columnas esten los datos de coordenadas:

In [46]:
df.head()

,ap01_01_Blanco,ap01_01_No_disponible,ap01_01_Multimarca,ap01_01_No_Corresponde,ap01_01_Enero,ap01_01_Febrero,ap01_01_Marzo,ap01_01_Abril,ap01_01_Mayo,ap01_01_Junio,...,ldesemp_Avanzado,ldesemp_Por_debajo_del_nivel_básico,ldesemp_Básico,ldesemp_Satisfactorio,cod_provincia,departamento,sector,ambito,lat,lon
0,38.100103,0.000000,7.106572,0.0,124.258442,127.471704,124.401839,127.723429,106.790121,129.026481,...,434.734796,329.761717,256.174319,612.420663,Ciudad Autónoma de Buenos Aires,Comuna 1,Estatal,Urbano,-34.609558,-58.388790
1,12.823730,9.769051,1.917169,0.0,109.102072,108.551388,99.718282,127.275887,125.298866,129.640379,...,553.412811,101.720018,139.053569,587.772806,Ciudad Autónoma de Buenos Aires,Comuna 1,Privado,Urbano,-34.609558,-58.388790
2,10.834863,1.979891,0.000000,0.0,74.520555,85.707571,101.649904,54.799012,60.765202,76.470780,...,176.431993,116.290074,174.814780,428.465844,Ciudad Autónoma de Buenos Aires,Comuna 10,Estatal,Urbano,-34.627458,-58.503098
3,15.528509,2.450926,1.604042,0.0,104.881607,103.273864,89.211479,156.139603,75.809453,79.649385,...,378.052771,86.016384,154.661004,563.029198,Ciudad Autónoma de Buenos Aires,Comuna 10,Privado,Urbano,-34.627458,-58.503098
4,4.037436,2.322993,0.000000,0.0,48.054819,40.722596,42.911736,63.165032,70.551349,48.007761,...,110.159428,96.242425,122.404094,308.268201,Ciudad Autónoma de Buenos Aires,Comuna 11,Estatal,Urbano,-34.603006,-58.500717


In [49]:
#Rellenamos las coordenadas vacias

def rellenar_coord_provincia(df):
    """
    Rellena las coordenadas vacías (lat/lon) usando la coordenada de la provincia.
    Guarda y usa cache (cache_geocoding.json).
    """
    geolocator = Nominatim(user_agent="mi_app_geocoding")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2)

    # Cargar cache si existe
    try:
        with open('cache_geocoding.json', 'r') as f:
            cache = json.load(f)
    except FileNotFoundError:
        cache = {}

    def obtener_coord_provincia(juris):
        if juris in cache:
            return cache[juris]
        try:
            loc = geocode(f"{juris}, Argentina")
            if loc:
                coords = [loc.latitude, loc.longitude]
                cache[juris] = coords
                return coords
            else:
                cache[juris] = [None, None]
                return [None, None]
        except:
            cache[juris] = [None, None]
            return [None, None]

    # Iterar sobre filas donde falten coordenadas
    for idx, row in df[df['lat'].isna() | df['lon'].isna()].iterrows():
        coords = obtener_coord_provincia(row['cod_provincia'])
        df.loc[idx, ['lat', 'lon']] = coords

    # Guardar cache actualizado
    with open('cache_geocoding.json', 'w') as f:
        json.dump(cache, f)

    return df

In [50]:
df = rellenar_coord_provincia(df)

In [51]:
guardar_df(df, ruta_archivo_final_latlon)

Archivo guardado exitosamente en: ../data/interim/2019_Ap_56latlon.csv


# Codificacion

* la columna sector y ambito van a codificarse de manera binaria

In [52]:
# Codificacion de sector y ambito

# Convertir: 
df['sector_binario'] = df['sector'].map({'Estatal': 1, 'Privado': 0})
df['ambito_binario'] = df['ambito'].map({'Urbano': 1, 'Rural': 0})

# Eliminacion de columnas:

* se eliminan columnas anteriores a la transformacion
- jurisdiccion, departamento, sector y ambito


* vamos a eliminar las columnas de indices, ya que estas estan formuladas
en base a la forma de responder de la encuesta o los niveles de educacion de los tutores.
- se eliminan las columnas asociadas a clima (clima escolar)
- se eliminan las columnas asociadas a NSE (nivel socio economico) 

In [58]:

pd.set_option('display.max_rows', None)
df.columns

Index(['ap01_01_Blanco', 'ap01_01_No_disponible', 'ap01_01_Multimarca',
       'ap01_01_No_Corresponde', 'ap01_01_Enero', 'ap01_01_Febrero',
       'ap01_01_Marzo', 'ap01_01_Abril', 'ap01_01_Mayo', 'ap01_01_Junio',
       ...
       'ldesemp_Básico', 'ldesemp_Satisfactorio', 'cod_provincia',
       'departamento', 'sector', 'ambito', 'lat', 'lon', 'sector_binario',
       'ambito_binario'],
      dtype='object', length=1420)

In [60]:
for c in df.columns:
    print(c)

ap01_01_Blanco
ap01_01_No_disponible
ap01_01_Multimarca
ap01_01_No_Corresponde
ap01_01_Enero
ap01_01_Febrero
ap01_01_Marzo
ap01_01_Abril
ap01_01_Mayo
ap01_01_Junio
ap01_01_Julio
ap01_01_Agosto
ap01_01_Septiembre
ap01_01_Octubre
ap01_01_Noviembre
ap01_01_Diciembre
ap01_02_Blanco
ap01_02_No_disponible
ap01_02_Multimarca
ap01_02_No_Corresponde
ap01_02_1998
ap01_02_1999
ap01_02_2000
ap01_02_2001
ap01_02_2002
ap01_02_2003
ap02_Blanco
ap02_No_disponible
ap02_Multimarca
ap02_Varón
ap02_Mujer
ap03_Blanco
ap03_No_disponible
ap03_Multimarca
ap03_Argentina
ap03_Bolivia
ap03_Brasil
ap03_Chile
ap03_Paraguay
ap03_Perú
ap03_Uruguay
ap03_Colombia
ap03_Venezuela
ap03_México
ap03_Otro_país_de_América
ap03_País_de_Europa
ap03_País_de_Asia
ap03_País_de_África
ap03_País_de_Oceanía
ap04_Blanco
ap04_No_disponible
ap04_Multimarca
ap04_Argentina
ap04_Bolivia
ap04_Brasil
ap04_Chile
ap04_Paraguay
ap04_Perú
ap04_Uruguay
ap04_Colombia
ap04_Venezuela
ap04_México
ap04_Otro_país_de_América
ap04_País_de_Europa
ap04_Pa

In [62]:
A_borrar = [
    'iinfraestructura_Estu_de_CUES_no_participantes_en_directivos',
'iinfraestructura_Estu_sin_índ_no_resp_en_todas_las_var_directi',
'iinfraestructura_Bajo',
'iinfraestructura_Medio',
'iinfraestructura_Alto',
'NSE_quintiles_Q1',
'NSE_quintiles_Q2',
'NSE_quintiles_Q3',
'NSE_quintiles_Q4',
'NSE_quintiles_Q5',
           "cod_provincia","departamento","sector","ambito"]


df.drop(columns=A_borrar, inplace=True)

In [63]:
def revisar_nans(df):
    """
    Verifica si un DataFrame tiene valores NaN y los imprime.
    """
    if df.isna().any().any():  # True si hay al menos un NaN
        print("El DataFrame tiene valores NaN.")
        # Opcional: mostrar cuántos NaN por columna
        #print(df.isna().sum())
    else:
        print("El DataFrame no tiene valores NaN.")
revisar_nans(df)

El DataFrame tiene valores NaN.


In [64]:
def mostrar_filas_nan(df):
    """
    Imprime las filas que contienen al menos un valor NaN en el DataFrame.
    """
    filas_nan = df[df.isna().any(axis=1)]
    if not filas_nan.empty:
        print("Filas con NaN:")
        print(filas_nan)
    else:
        print("No hay filas con NaN.")
mostrar_filas_nan(df)

Filas con NaN:
      ap01_01_Blanco  ap01_01_No_disponible  ap01_01_Multimarca  \
686              0.0                    0.0                 0.0   
851              0.0                    0.0                 0.0   
1038             0.0                    0.0                 0.0   

      ap01_01_No_Corresponde  ap01_01_Enero  ap01_01_Febrero  ap01_01_Marzo  \
686                      0.0            0.0              0.0            0.0   
851                      0.0            0.0              0.0            0.0   
1038                     0.0            0.0              0.0            0.0   

      ap01_01_Abril  ap01_01_Mayo  ap01_01_Junio  ...  mdesemp_Satisfactorio  \
686             0.0           0.0            2.0  ...                    NaN   
851             0.0           0.0            0.0  ...                    NaN   
1038            0.0           0.0            0.0  ...                    NaN   

      mdesemp_Avanzado  ldesemp_Avanzado  ldesemp_Por_debajo_del_nivel_básico 

In [ ]:
# Hay 3 filas NAN en 2019

In [65]:
# eliminar filas que tengan al menos un NaN
df = df.dropna()

In [66]:
revisar_nans(df)

El DataFrame no tiene valores NaN.


# Listo

Ya tenemos un dataset limpio para el año 2024.
La idea de este notebook es utilizarlo para todos los años restantes y hacer una union entre todos

In [67]:
guardar_df(df, ruta_archivo_final_final)

Archivo guardado exitosamente en: ../data/processed/2019_Ap_56final.csv
